In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gym

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
def action_simple(observation):
    """Move left if pole is leaning left and vice versa"""
    if observation[2] < 0:
        return 0
    else:
        return 1

In [ ]:
def action_sigmoid(observation, theta):
    """"""
    

In [ ]:
success_array = np.zeros(1)

for episode in range(success_array.shape[0]):
    observation = env.reset()
    
    # Do 100 steps and note when the pole fell
    for t in range(100):
        action = action_simple(observation)
        observation, reward, done, info = env.step(action)
        if done:
            success_array[episode] = t + 1
            break
            
# Average time before the pole falls
print(np.mean(success_array))

In [ ]:
"""Results over 10000 episodes:

Action                     Mean time until done
constant (0 or 1)          9.35
action_simple              42.17

"""

In [ ]:
theta = tf.placeholder(tf.float32, shape=(4,))
state = tf.placeholder(tf.float32, shape=(4,))
dot = tf.tensordot(theta, state, axes=1)
sigmoid = tf.nn.sigmoid(dot)

In [ ]:
with tf.Session() as sess:
    feed_dict = {theta: np.array([1, 1, 1, 1]), \
                 state: np.array([1, 1, 1, 1])}
    out = sess.run(sigmoid, feed_dict=feed_dict)
    print(out)